In [1]:
import os
os.chdir("../..")

In [2]:
import torch
from utils.dataset_loader import load_datasets
from utils.model_utils import initialize_model
from utils.train_utils import train_model
from utils.metrics import evaluate_model
from utils.visualization import plot_training, plot_confusion_matrix

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"[INFO] Using device: {device}")

[INFO] Using device: cuda


In [4]:
data_dir = "wildfire_dataset_scaled"
batch_size = 32
num_epochs = 10
learning_rate = 0.001
num_classes = 2

In [5]:
print("[INFO] Loading augmented datasets...")
train_loader, val_loader, test_loader = load_datasets(data_dir, batch_size, augmentation="augmented")
print("[INFO] Augmented datasets loaded successfully!")



[INFO] Loading augmented datasets...
[INFO] Augmented datasets loaded successfully!


In [6]:
print("\n[INFO] Starting Training for Model: vgg16 with Augmented Data\n")
print("[INFO] Initializing VGG16 model...")
model = initialize_model(
    model_name="vgg16",
    num_classes=num_classes,
    pretrained=True,
    freeze_all=True  # Augmented data: still freeze all layers
)
print("[INFO] Model initialized successfully!")


[INFO] Starting Training for Model: vgg16 with Augmented Data

[INFO] Initializing VGG16 model...


c:\Users\Yash\Documents\Development\Projects\final_year\env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Yash\Documents\Development\Projects\final_year\env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[INFO] Model initialized successfully!


In [7]:
print("[INFO] Setting up loss function and optimizer...")
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate)
print("[INFO] Loss function and optimizer set up!")

[INFO] Setting up loss function and optimizer...
[INFO] Loss function and optimizer set up!


In [ ]:
print("[INFO] Starting model training with augmented data...")
history = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=None,  # No learning rate scheduler for this stage
    num_epochs=num_epochs,
    device=device
)
print("[INFO] Training completed successfully!")

[INFO] Starting model training with augmented data...

Starting training...



Epoch [1/10] - Training:  42%|████▏     | 25/59 [00:24<00:32,  1.04it/s, Batch Loss=0.4899]

In [ ]:
# Evaluate Model
print("[INFO] Evaluating the model...")
metrics = evaluate_model(
    model=model,
    test_loader=test_loader,
    classes=["No Fire", "Fire"],
    device=device
)
print(f"\n[INFO] Metrics for VGG16 with Augmented Data:\n{metrics}")

In [ ]:
# Save Plots
print("[INFO] Saving training and evaluation results...")
plot_training(history, "outputs/vgg16_augmented_training_curve.png")
plot_confusion_matrix(
    cm=metrics["confusion_matrix"],
    classes=["No Fire", "Fire"],
    output_path="outputs/vgg16_augmented_confusion_matrix.png"
)
print("[INFO] Results saved successfully!")